In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Установка необходимых библиотек

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets

Импортирование необходимых библиотек

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, TrainingArguments, Trainer
from datasets import load_dataset

Подготовка данных

In [ ]:
!python prepare_messages.py --tg-history-path data.json --output-path data.csv

In [ ]:
import pandas as pd

data = pd.read_csv('data.csv')

In [ ]:
data

,context_3,context_2,context_1,response
0,NaN,NaN,NaN,Ору\nЗачем\nЧто
1,NaN,NaN,Ору\nЗачем\nЧто,На всякий
2,NaN,NaN,NaN,Что мне читать теперь?
3,NaN,NaN,Что мне читать теперь?,Вместе смеёмся
4,NaN,NaN,NaN,Ору
...,...,...,...,...
2597,А учеба,Ну я тут не могу ничего сделать 😁,Из-за того что у паспорта срок заканчивается п...,"Да, мне нужно было сделать гражданство, а я не..."
2598,Ну я тут не могу ничего сделать 😁,Из-за того что у паспорта срок заканчивается п...,"Да, мне нужно было сделать гражданство, а я не...","😍\nРуслан, привет)\nТы в порядке?\nКуда-то пропал"
2599,Из-за того что у паспорта срок заканчивается п...,"Да, мне нужно было сделать гражданство, а я не...","😍\nРуслан, привет)\nТы в порядке?\nКуда-то пропал","Привет, все хорошо, я не пропал , занимаюсь до..."
2600,"Да, мне нужно было сделать гражданство, а я не...","😍\nРуслан, привет)\nТы в порядке?\nКуда-то пропал","Привет, все хорошо, я не пропал , занимаюсь до...",Пропал уж😅\nНормально все с документами?


Формирование тестовых и валидационных данных

In [ ]:
data_train = data.dropna(axis=0, subset=['context_1']).fillna('')
data_train['dialog'] = data_train.apply(' '.join, axis=1)

data_val = pd.DataFrame(data[data['context_1'].isna() == True]['response']).rename(columns={'response': 'dialog'}).dropna()

# сохранение сконкатенированных данных
data_train.to_csv('data_train.csv', columns = ['dialog'], index=False)
data_val.to_csv('data_val.csv', columns = ['dialog'], index=False)

Скачивание токенайзера и модели ruDialogGPT-medium

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')
model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1468: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Формирование датасета

In [ ]:
data_files = {'train':"data_train.csv", 'validation': 'data_val.csv'}
dataset = load_dataset("csv", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Токенизация слов

In [ ]:
def encode(examples):
    encoded = tokenizer(examples['dialog'], truncation=True, padding='max_length', max_length=128)
    encoded['labels'] = encoded['input_ids'][:]
    return encoded

encoded_dataset = dataset.map(encode, batched=True)

Map:   0%|          | 0/1691 [00:00<?, ? examples/s]

Map:   0%|          | 0/910 [00:00<?, ? examples/s]

In [ ]:
# Выбор девайса
device = 'cuda' if torch.cuda.is_available else 'cpu'

# Определение аргументов для обучения
training_args = TrainingArguments(
    output_dir='arguments',   # output directory
    num_train_epochs=10,             # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=None,                # directory for storing logs
    fp16=True                        # use floating point 16 bit precision for training
)

# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation']
)

Оценка модели на нашем датасете до fine-tuning

In [ ]:
# Evaluate before fine-tuning
pre_eval_results = trainer.evaluate(encoded_dataset['validation'])

# Get predictions for validation set before fine tuning for 10 samples
pre_val_predictions = trainer.predict(encoded_dataset['validation'].select(range(10)))

Fine-tuning модели

In [ ]:
# Fine-tune the model
trainer.train()

Step,Training Loss
500,0.321500
1000,0.168900
1500,0.089300
2000,0.062300


TrainOutput(global_step=2120, training_loss=0.15456842701390106, metrics={'train_runtime': 1042.2744, 'train_samples_per_second': 16.224, 'train_steps_per_second': 2.034, 'total_flos': 3926082186117120.0, 'train_loss': 0.15456842701390106, 'epoch': 10.0})

Оценка модели после fine-tuning и сравнение результатов.

In [ ]:
# Evaluate after fine-tuning
post_eval_results = trainer.evaluate(encoded_dataset['validation'])

# Print the evaluation losses before and after fine-tuning
print('Evaluation Results before fine-tuning :', pre_eval_results['eval_loss'])
print('Evaluation Results after fine-tuning  :', post_eval_results['eval_loss'])

# Get predictions for validation set before fine tuning for 10 samples
post_val_predictions = trainer.predict(encoded_dataset['validation'].select(range(10)))

# Zip the pre and post tuning predictions
predictions = zip(pre_val_predictions.predictions, post_val_predictions.predictions)

Evaluation Results before fine-tuning : 24.885087966918945
Evaluation Results after fine-tuning  : 0.4313463866710663
